In [2]:
#libary dependencies
!pip install hmmlearn #https://github.com/hmmlearn/hmmlearn

The directory '/home/cseos2g/datduyn/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/cseos2g/datduyn/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
Exception:
Traceback (most recent call last):
  File "/util/opt/anaconda/4.3.14/envs/jupyterhub-root/lib/python3.6/site-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/util/opt/anaconda/4.3.14/envs/jupyterhub-root/lib/python3.6/site-packages/pip/commands/install.py", line 342, in run
    prefix=options.prefix_path,
  File "/util/opt/anaconda/4.3.14/envs/jupyterhub-root/lib/python3.6/site-packages/pip/req/req_set.py", line 784, in install
 

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import os
from hmmlearn import hmm
from nltk import FreqDist
import json
# from sklearn.hmm import GaussianHMM

#utils
import pickle
def save_model(hmm_model, save_name='./hmm_model.pkl'):
    with open(save_name, 'wb') as output:
        pickle.dump(hmm_model, output, pickle.HIGHEST_PROTOCOL)
        
def load_model(dir_path='./'):
    model = None
    with open(dir_path+'/hmm_model.pkl', 'rb') as input:
        model = pickle.load(input)
    return model

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Character base music generation with HMM
## big pictures[2]

- **Generative**
    - Models P(x, y)
    - prediting using Bayes rule $argmax_Y P(y\|x)
    - HMM
    
- Discriminative
    - Models P(y|x)
    - predicting using $argmax_y P(y\|x)$
    - RNN

## Data and Features
Load data file contains abc notated tunes. Tunes are concatenated one after other in one file without abc notated headers.

In [2]:
DATA_PATH = './data/jig_hornpipes_cleaned.txt'
MODEL_DIR = '../hmm_model/04_24_19_hmm_multinomial_nvocab_components_init_emission_iter50'

#make sure path to model dir exist
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [3]:
file = open(DATA_PATH, mode = 'r')
data = file.read()
file.close()

In [4]:
#sanity check
print(data[:100])

f|"A"ecc c2f|"A"ecc c2f|"A"ecc c2f|"Bm"BcB "E7"B2f|
"A"ecc c2f|"A"ecc c2c/2d/2|"D"efe "E7"dcB| [1"A"


In [5]:
def get_data_info(data, save_path=None):
    #mapping character to index
    vocab_map = {}
    chars = list(set(data))
    vocab_map['char2idx'] = {ch: i for (i, ch) in enumerate(chars)}
    vocab_map['idx2char'] = {i: ch for (i, ch) in enumerate(chars)}
    print("Number of unique characters in our whole tunes database = {}".format(len(vocab_map['char2idx']))) #87
    
    if save_path:
        with open(save_path, mode = "w+") as f:
            json.dump(vocab_map, f)
        
    n_vocab = len(vocab_map['char2idx'])
    return vocab_map

def get_train_val(data, train_frac=0.8):
    '''
    -data: np array of shape (N,) each cell in arrary
    represent the character in idx
    '''
    n_train = int(len(data) * train_frac)
    X_train, X_val = data[:n_train], data[n_train:len(data)]
    return X_train, X_val
vocab_map = get_data_info(data, save_path=os.path.join(MODEL_DIR, 'model_dictionary.json'))
n_vocab = len(vocab_map['char2idx'])

Number of unique characters in our whole tunes database = 67


### Split data and convert to numpy numerical values

In [6]:
all_characters = np.asarray([vocab_map['char2idx'][c] for c in data], dtype = np.int64)
# Xy_train, Xy_val = get_train_val(all_characters, train_frac=0.8)

In [7]:
all_characters.shape

(83542,)

### Hidden markov model and its variation
#### Multinomial HMM model


### Train model on small data
- In this case, we only train on small dset for sannity check. We train on the full dset on crane refer to `./src/hmm_train.py` for more detail.

In [9]:
fd = FreqDist(all_characters)
frequencies = np.fromiter((fd.freq(i) for i in range(n_vocab)), dtype=np.float64)
emission_prob = np.stack([frequencies] * n_vocab)

model = hmm.MultinomialHMM(n_components=n_vocab, 
                        n_iter=2, verbose=True, tol=0.001)

model.emissionprob_ = emission_prob

In [10]:
# features = np.fromiter(batches, np.int64)
features = np.array([all_characters]).T

In [11]:
# verbose True: report 3 columns. iteration number, 
# log probability of the data at the current iteration and convergence rate
#try to overfit small data
# lengths = [len(line) for line in batches]
model.fit(features)

         1     -351107.4796             +nan
         2     -251660.4069      +99447.0726


MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=67,
        n_iter=2, params='ste',
        random_state=<mtrand.RandomState object at 0x2b46baaacd80>,
        startprob_prior=1.0, tol=0.001, transmat_prior=1.0, verbose=True)

## Save model

In [14]:
save_model(model)

## Model evaluation: Randomly sampling from likelyhood and transition matrixc

In [48]:
MODEL_DIR = './hmm_model/04_24_19_hmm_multinomial_nvocab_components_init_emission_right_iter50/'
hmm_model = load_model(dir_path=MODEL_DIR)

In [65]:
def sampling(model, vocab_map, n_samples=100):
    X, state = model.sample(n_samples)
    seq = ''.join(vocab_map['idx2char'][int(c)] for c in X)
    return seq
music_abc = sampling(model, vocab_map, n_samples=400)
print(music_abc)

G2f3Efb|A m"d"ae\fbc3E(cgGB"
"G2"bG"Gd"3c 3|" "|PBBegd2FDc"g|G32""""2"GM7fB3""" DG"2"A"c|dA"d^ " "e Be(f"mBdAmme BeAB"
g3|3A""|2
7Bd|gde3"de
7gBe"2 d"E|"2"  Bd" |A c A"\GdBB"V cG"""2m"EGa"E| 2f"G
c cD:d""
D"/eD7:fg" "2
"G#""|g(GDg| |2(7BB3  ed"C"A"GGA2""
 A""/f/:2 "#g"ceC"""""G|^2F"GB|22
2|BBA2""G"DeGd
DD"m |Ge "dGd"7""2/cdf FfF |2gcDG bB^d AAaA
" mg2"A"m"AGGG"""3|eB  |cEde"dBd/"A" " cd"7B|3gd"A G


### Quick Discussion on model prediction
- HMM model

In [60]:
from src.evaluation import save_midi
save_midi(music_abc, save_path='./midi.mid', print_trace=True)

failed parsing


Traceback (most recent call last):
  File "/lustre/work/cseos2g/datduyn/Documents/songnet/music_generation.python/src/evaluation.py", line 43, in save_midi
    abcScore = converter.parse(music_abc, format='abc')
  File "/home/cseos2g/datduyn/.conda/envs/cs478project/lib/python3.6/site-packages/music21/converter/__init__.py", line 1141, in parse
    return parseData(value, number=number, format=m21Format, **keywords)
  File "/home/cseos2g/datduyn/.conda/envs/cs478project/lib/python3.6/site-packages/music21/converter/__init__.py", line 1017, in parseData
    v.parseData(dataStr, number=number, format=format, **keywords)
  File "/home/cseos2g/datduyn/.conda/envs/cs478project/lib/python3.6/site-packages/music21/converter/__init__.py", line 610, in parseData
    self.subConverter.parseData(dataStr, number=number)
  File "/home/cseos2g/datduyn/.conda/envs/cs478project/lib/python3.6/site-packages/music21/converter/subConverters.py", line 1016, in parseData
    abcHandler = af.readstr(strData,

False

## Model Evaluation: Sampling given starting seed(starting oservable sequence)[3]

## Understandning model prediction

# references
[1] (ML 16.6) Gaussian mixture model (Mixture of Gaussians). Available at https://www.youtube.com/watch?v=Rkl30Fr2S38. 2011

[2] HMM Lecture slides. available at https://www.cs.cmu.edu/~epxing/Class/10701-10s/recitation/recitation6.pdf

[3] Predicting next given current state. available at https://github.com/hmmlearn/hmmlearn/issues/171